In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from datetime import datetime
from math import ceil

In [5]:
start = datetime(2022, 12, 27, 8, 44)
test_duration = np.timedelta64(8, 'h')

def to_delta(df, column):
    print(start)
    df[column] = df[column].apply(lambda dt: dt - start)
    df.set_index(column, inplace=True, drop=False)

def load_cloudwatch(filename):
    logs = pd.read_csv(filename)
    logs = logs.sort_values(by='timestamp')
    logs['timestamp'] = pd.to_datetime(logs['timestamp'])
    logs = logs[logs['timestamp'] > start]
    to_delta(logs, 'timestamp')
    return logs

def start_plot():
    fig, ax = plt.subplots()
    ax.set_xlabel("Czas trwania testu [h]")

    return (fig, ax)

def human_delta(index):
    return index.get_level_values('timestamp').seconds / 60 / 60

In [14]:
prefix = 'serverless_v4_raw_data/'
filename = f"{prefix}/logs.csv"

# https://aws.amazon.com/lambda/pricing/, Frankfurt, 2048 MB
price_per_ms = 0.0000000333


logs = load_cloudwatch(filename)
logs['billedDuration'] = logs['duration'].apply(lambda d: ceil(d))
logs.count(), logs.sum()['billedDuration'] * price_per_ms * 30

2022-12-27 08:44:00


(timestamp         9294
 duration          9294
 maxMemoryUsed     9294
 memorySize        9294
 initDuration        17
 billedDuration    9294
 dtype: int64,
 0.461898639)

In [18]:
def load_locust(filename):
    df =  pd.read_csv(filename)
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')
    start = df['Timestamp'][0]
    df['Timestamp'] = df['Timestamp'].apply(lambda dt: dt - start)
    df.set_index('Timestamp', inplace=True, drop=False)
    return df

serverless = load_locust('../load-tests/results/serverless_v4_stats_history.csv')
serverless[serverless['Name'] == '/upload-photo']

,Timestamp,User Count,Type,Name,Requests/s,Failures/s,50%,66%,75%,80%,...,99.9%,99.99%,100%,Total Request Count,Total Failure Count,Total Median Response Time,Total Average Response Time,Total Min Response Time,Total Max Response Time,Total Average Content Size
Timestamp,,,,,,,,,,,,,,,,,,,,,
0 days 00:44:31,0 days 00:44:31,21,PUT,/upload-photo,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4,0,610,625.250000,581,698,0.0
0 days 00:44:32,0 days 00:44:32,21,PUT,/upload-photo,0.0,0.0,620.0,620.0,700.0,700.0,...,700.0,700.0,700.0,4,0,610,625.250000,581,698,0.0
0 days 00:44:33,0 days 00:44:33,21,PUT,/upload-photo,0.0,0.0,620.0,620.0,700.0,700.0,...,700.0,700.0,700.0,4,0,610,625.250000,581,698,0.0
0 days 00:44:34,0 days 00:44:34,21,PUT,/upload-photo,0.0,0.0,620.0,620.0,700.0,700.0,...,700.0,700.0,700.0,4,0,610,625.250000,581,698,0.0
0 days 00:44:35,0 days 00:44:35,21,PUT,/upload-photo,0.0,0.0,620.0,620.0,700.0,700.0,...,700.0,700.0,700.0,4,0,610,625.250000,581,698,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0 days 07:59:54,0 days 07:59:54,0,PUT,/upload-photo,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,23,0,610,615.434783,268,785,0.0
0 days 07:59:55,0 days 07:59:55,0,PUT,/upload-photo,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,23,0,610,615.434783,268,785,0.0
0 days 07:59:56,0 days 07:59:56,0,PUT,/upload-photo,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,23,0,610,615.434783,268,785,0.0


In [23]:
max_put_requests = serverless[serverless['Name'] == '/upload-photo']['Total Request Count'][-1] * 30
max_get_requests = serverless[serverless['Name'] == '/download-photo']['Total Request Count'][-1] * 30
max_size = max_get_requests * 15 / 1000

max_put_requests, max_get_requests, max_size

(690, 3628620, 54429.3)